# NBA Player Salary Prediction - Linear Regression

Tujuan dari project ini adalah untuk memprediksi gaji pemain NBA berdasarkan atribut data yang mereka miliki. Kasus ini termasuk permasalahan regresi pada Supervised Learning.

Sumber data berasal dari [Kaggle](https://www.kaggle.com/code/dionisiusdh/nba2k20-data-visualization/data). Dataset ini merupakan data NBA 2K20 Rating pada tahun 2020, yang terdiri dari 20 kolom antara lain:
1. Unnamed: 0
2. full_name
3. rating
4. jersey
5. team
6. position
7. b_day
8. height_in_m
9. weight_in_kg
10. salary
11. country
12. draft_year
13. draft_round
14. draft_peak
15. college
16. current_year
17. year_played
18. body_mass_index
19. bmi_class
20. attend_college

Kita akan mengestimasi gaji seorang pemain pada tahun 2020 dan 2022 (sudah diketahui) sehingga kita akan tahu seberapa besar kesalahan estimasi tersebut. Kemudian kita akan memprediksi gaji pemain tersebut pada tahun 2024 (yang belum diketahui saat ini). Dengan skenario ini, ada tiga langkah proses yangb akan kita lakukan:
* Membagi dataset menjadi training set dan test set. Pada studi kasus kali ini, kita akan menggunakan empat jenis algoritma Regresi Linear dan memilih salah satu algoritma dengan kinerja terbaik terhadap test set.
* Setelah algoritma terbaik didapat, kita akan membuat model akhir dengan algoritma tersebut. Pada pelatihan model akhir, kita menggunakan seluruh dataset yang bertujuan memanfaatkan secara maksimal dataset untuk membuat model yang akan digunakan pada dunia nyata, yaitu pada kasus-kasus yang belum pernah dilihat sebelumnya.
* Dari model akhir yang telah dibuat, kita akan menggunakannya pada kasus yang nyata. Disini kita akan menghitung gaji LeBron James (sudah dihapus sebelumnya) pada tahun 2020 dan 2022, kemudian kita akan memprediksi gajinya nanti pada tahun 2024.

Let's do it!

In [ ]:
import pandas as pd

# Baca dataset
df = pd.read_csv('/content/NBA-2K.csv')
df.head()

,Unnamed: 0,full_name,rating,jersey,team,position,b_day,height_in_m,weight_in_kg,salary,country,draft_year,draft_round,draft_peak,college,current_age,year_played,body_mass_index,bmi_class,attended_college
0,1,Kawhi Leonard,97,#2,Los Angeles Clippers,F,6/29/1991,2.01,102.1,32742000,USA,2011,1,15,San Diego State,29,9,25.3,Overweight,0
1,2,Giannis Antetokounmpo,96,#34,Milwaukee Bucks,F-G,12/6/1994,2.11,109.8,25842697,Greece,2013,1,15,NaN,26,7,24.7,Normal,1
2,3,Kevin Durant,96,#7,Brooklyn Nets,F,9/29/1988,2.08,104.3,37199000,USA,2007,1,2,Texas,32,13,24.1,Normal,0
3,4,James Harden,96,#13,Houston Rockets,G,8/26/1989,1.96,99.8,38199000,USA,2009,1,3,Arizona State,31,11,26.0,Overweight,0
4,5,Stephen Curry,95,#30,Golden State Warriors,G,3/14/1988,1.91,83.9,40231758,USA,2009,1,7,Davidson,33,12,23.0,Normal,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        428 non-null    int64  
 1   full_name         428 non-null    object 
 2   rating            428 non-null    int64  
 3   jersey            428 non-null    object 
 4   team              405 non-null    object 
 5   position          428 non-null    object 
 6   b_day             428 non-null    object 
 7   height_in_m       428 non-null    float64
 8   weight_in_kg      428 non-null    float64
 9   salary            428 non-null    int64  
 10  country           428 non-null    object 
 11  draft_year        428 non-null    int64  
 12  draft_round       428 non-null    int64  
 13  draft_peak        428 non-null    int64  
 14  college           363 non-null    object 
 15  current_age       428 non-null    int64  
 16  year_played       428 non-null    int64  
 1

Dataset terdiri dari 20 kolom. Kolom "salary" merupakan label yang akan menjadi acuan dalam pelatihan model. Perhatikan kolom yang bertipe numerik.

In [ ]:
df.describe()

,Unnamed: 0,rating,height_in_m,weight_in_kg,salary,draft_year,draft_round,draft_peak,current_age,year_played,body_mass_index,attended_college
count,428.000000,428.000000,428.000000,428.000000,4.280000e+02,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000
mean,214.500000,76.324766,1.995491,97.182009,8.462541e+06,2014.119159,1.088785,18.247664,27.123832,6.200935,24.371262,0.151869
std,123.697211,5.575510,0.083554,10.519589,9.121959e+06,3.905477,0.628418,16.048941,4.087004,3.908413,1.783476,0.359314
min,1.000000,67.000000,1.750000,77.100000,5.000000e+04,2001.000000,0.000000,0.000000,20.000000,1.000000,20.300000,0.000000
25%,107.750000,72.000000,1.930000,89.400000,2.000000e+06,2012.000000,1.000000,4.000000,24.000000,3.000000,23.100000,0.000000
50%,214.500000,75.000000,2.010000,96.600000,4.310160e+06,2015.000000,1.000000,14.500000,27.000000,6.000000,24.200000,0.000000
75%,321.250000,79.000000,2.060000,104.300000,1.211977e+07,2017.000000,1.000000,30.000000,30.000000,9.000000,25.500000,0.000000
max,428.000000,97.000000,2.240000,131.500000,4.023176e+07,2019.000000,2.000000,60.000000,40.000000,19.000000,32.900000,1.000000


Pada studi kasus kali ini, dari kolom-kolom yang bertipe numerik akan kita pilih 5 kolom utama untuk pelatihan model.

In [ ]:
# Ambil input variable yang diperlukan
col = ['rating', 'height_in_m', 'weight_in_kg', 'current_age', 'year_played']

X = df[col]
y = df['salary']

In [ ]:
X.head()

,rating,height_in_m,weight_in_kg,current_age,year_played
0,97,2.01,102.1,29,9
1,96,2.11,109.8,26,7
2,96,2.08,104.3,32,13
3,96,1.96,99.8,31,11
4,95,1.91,83.9,33,12


In [ ]:
X.describe()

,rating,height_in_m,weight_in_kg,current_age,year_played
count,428.000000,428.000000,428.000000,428.000000,428.000000
mean,76.324766,1.995491,97.182009,27.123832,6.200935
std,5.575510,0.083554,10.519589,4.087004,3.908413
min,67.000000,1.750000,77.100000,20.000000,1.000000
25%,72.000000,1.930000,89.400000,24.000000,3.000000
50%,75.000000,2.010000,96.600000,27.000000,6.000000
75%,79.000000,2.060000,104.300000,30.000000,9.000000
max,97.000000,2.240000,131.500000,40.000000,19.000000


In [ ]:
X.shape, y.shape

((428, 5), (428,))

Selanjutnya, lakukan pemecahan data dengan 80% untuk training set dan 20% untuk test set.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Bagi data menjadi 80% training set and 20% test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
len(X_train), len(y_train)

(342, 342)

In [ ]:
X_train.tail()

,rating,height_in_m,weight_in_kg,current_age,year_played
71,81,1.98,128.8,20,1
106,79,2.06,102.1,30,10
270,74,2.11,99.8,20,1
348,72,2.11,113.4,21,1
102,79,2.13,120.2,33,13


Buat model Regresi Linear dengan empat algoritme pembelajaran yang berbeda. Selain library LinearRegression, kita juga akan menggunakan library RidgeCV, LassoCV, dan ElasticNetCV.

In [ ]:
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV

In [ ]:
model1 = LinearRegression() #buat model regresi linier
model2 = RidgeCV(alphas=(.1, 1, 10, 100), scoring='neg_mean_absolute_error')
model3 = LassoCV(eps=0.001, alphas=[10, 50, 100, 150, 200], cv=5)
model4 = ElasticNetCV(l1_ratio=[.1, .2, .3, .4, .5, .6, .7, .8, .9, 1.], cv=5)

Lakukan pelatihan model dengan training set.

In [ ]:
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)
model3.fit(X_train, y_train)
model4.fit(X_train, y_train)

ElasticNetCV(cv=5, l1_ratio=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

Lakukan evaluasi kinerja masing-masing model menggunakan test set.

In [ ]:
# Prediksi semua data dengan model yang telah dibuat
pred1 = model1.predict(X_test)
pred2 = model2.predict(X_test)
pred3 = model3.predict(X_test)
pred4 = model4.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
# Hitung MAE dari hasil prediksi dengan model yang telah dibuat
MAE1 = mean_absolute_error(y_test, pred1)
MAE2 = mean_absolute_error(y_test, pred2)
MAE3 = mean_absolute_error(y_test, pred3)
MAE4 = mean_absolute_error(y_test, pred4)

print(f'Model 1: {MAE1}')
print(f'Model 2: {MAE2}')
print(f'Model 3: {MAE3}')
print(f'Model 4: {MAE4}')

Model 1: 5137975.589567243
Model 2: 5117719.175194766
Model 3: 5137837.775409552
Model 4: 5132061.613065206


Dari hasil yang didapat, model4 mendapatkan nilai MAE terkecil (optimal) maka kita akan menggunakan ElasticNetCV (model4) untuk pembuatan model akhir.

In [ ]:
df.salary.mean()

8462541.072429907

##Build Final Model

Buat model akhir dengan seluruh dataset yang tersedia.

In [ ]:
model4.fit(X, y)

ElasticNetCV(cv=5, l1_ratio=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

Kita akan melihat bobot-bobot dari variable input sesuai dengan urutan kolom (indeks kolom) menggunakan fungsi .coef_

In [ ]:
model4.coef_

array([1135757.87150715,       0.        ,  -39394.93868392,
        264980.89036588,  304120.38497236])

Kita juga bisa melihat bias dari hasil pembelajaran model dengan menggunakan fungsi .intercept_

In [ ]:
model4.intercept_

-83468561.51255384

Mari kita lihat bobot dan bias dari hasil pembelajaran model menggunakan tabel. Kita akan menggunakan fungsi pd.DataFrame() untuk membuatnya.

In [ ]:
pd.DataFrame(data=model4.coef_, index=X.columns, columns=['Coeff'])

,Coeff
rating,1.135758e+06
height_in_m,0.000000e+00
weight_in_kg,-3.939494e+04
current_age,2.649809e+05
year_played,3.041204e+05


Dari hasil yang didapatkan, terlihat variable "tinggi" atau height_in_m bernilai nol sehingga variable ini tidak akan digunakan dalam memprediksi gaji.

Selanjutnya adalah saatnya kita memprediksi gaji LeBron James tahun 2020 dan 2022, yang menurut catatan sebesar `$37.436.858` dan `$41.180.000`

In [ ]:
# Gaji aktual
salary = 37436858 # in 2020

# Kondisi atribut data LeBron James tahun 2020
# rating, tinggi, berat, umur dan pengalaman
james2020 = [[97, 2.06, 113.4, 36, 17]]

# Prediksi gaji di 2020
pred = model4.predict(james2020)

print(f'estimasi gaji: {pred} dengan kesalahan estimasi: {pred-salary}')

estimasi gaji: [36941924.57458478] dengan kesalahan estimasi: [-494933.42541522]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but ElasticNetCV was fitted with feature names
  warnings.warn(


In [ ]:
# Gaji aktual
salary = 41180000 # in 2022

# Kondisi atribut data LeBron James tahun 2022
# rating, tinggi, berat, umur dan pengalaman
james2022 = [[97, 2.06, 113.4, 38, 19]]

# Prediksi gaji di 2022
pred = model4.predict(james2022)

print(f'estimasi gaji: {pred} dengan kesalahan estimasi: {pred-salary}')

estimasi gaji: [38080127.12526126] dengan kesalahan estimasi: [-3099872.87473874]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but ElasticNetCV was fitted with feature names
  warnings.warn(


In [ ]:
# Kondisi atribut data LeBron James tahun 2024
# rating, tinggi, berat, umur dan pengalaman
james2024 = [[97, 2.06, 113.4, 40, 21]]

# Prediksi gaji di 2024
pred = model4.predict(james2024)

print(pred)

[39218329.67593774]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but ElasticNetCV was fitted with feature names
  warnings.warn(
